In [6]:
import joblib
import praw
import data_scrapping as ds
from urllib.parse import urlparse
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
import nltk
import xgboost
from sklearn.model_selection import train_test_split
import cgi

In [7]:
def vectorisation(train, test):
    vect = CountVectorizer()
    
    new_train = vect.fit_transform(train)
    new_test = vect.transform(test)
    return new_train, new_test

In [8]:
url_to_check = input()
reddit = praw.Reddit(client_id = '#', \
                                client_secret = '#',\
                                user_agent = '#',\
                                username = '#',\
                                password = '#')

post = reddit.submission(url = url_to_check)
title_c = ds.process_strings(str(post.title))
content_c = ds.process_strings(str(post.selftext))

post.comments.replace_more(limit=0)
comments = ""
for comment in post.comments.list():
    comments = comments + comment.body

comments = ds.process_strings(comments)

comments_c = ds.process_strings(comments)

url = post.url
parsed = urlparse(url)
url_path_c = ds.process_strings(" ".join(re.split('/|-|_', str(parsed.path))))

https://www.reddit.com/r/india/comments/cg82rw/weekly_financial_advice_thread_july_22_2019/


In [10]:
insert = {
    "title" : [title_c],
    "content" : [content_c],
    "comments" : [comments_c],
    "url_path" : [url_path_c]
}
df = pd.DataFrame(insert)
df

,title,content,comments,url_path
0,weekly financial advice thread july,weekly thread everything related indian bankin...,fresher start investing first salary onwards m...,r india comments weekly financial advice threa...


In [11]:
df["merged"] = df["title"] + df["url_path"] + df["comments"] + df["content"]
df.merged

0    weekly financial advice thread julyr india com...
Name: merged, dtype: object

In [12]:
collection_pointer = ds.initialising_mongo()
complete_dataset = pd.DataFrame(list(collection_pointer.find()))
complete_dataset.dropna(inplace = True)
complete_dataset['merged'] = complete_dataset["title"] + complete_dataset["url_path"] + complete_dataset['comments'] + complete_dataset["content"]

Y = complete_dataset.flair
x = complete_dataset.drop('flair', axis = 1)

x_train, x_test, y_train, y_test = train_test_split(x, Y, test_size = 0.2, random_state = 7)

In [13]:
a,fn = vectorisation(x_train.merged, df.merged)
trained_model = joblib.load("flair_predictor.joblib")
# print(trained_model)
trained_model.predict(fn)

array(['AskIndia'], dtype=object)